In [38]:
import pandas as pd

### Read Expected and Obesrved subtitutions and take only CytB and 3 Position

In [39]:
obs_mut = pd.read_csv('../data/ObsMutSpec.csv')
obs_mut = obs_mut[obs_mut['MutType'] >= 1]
obs_mut = obs_mut[obs_mut['Gene'] == 'CytB']
exp_mut = pd.read_csv('../data/exp_mut_spec.csv',sep='\t')
exp_mut = exp_mut[exp_mut['MutType'] >= 1]
exp_mut = exp_mut[exp_mut['Gene'] == 'CYTB']

### Check how many species we have in each data set

In [40]:
sps_exp = exp_mut['Species']
sps_obs = obs_mut['Species']
print(len(sps_exp.unique()), len(sps_obs.unique()), sep=', ')


1312, 1562


### Take species that presented in each dataset 

In [41]:
obs_mut = obs_mut.loc[obs_mut['Species'].isin(sps_exp.unique())]
sps_obs = obs_mut['Species']
exp_mut = exp_mut.loc[exp_mut['Species'].isin(sps_obs.unique())]

### Count codons in each gene for each species for observed and expected mutations

In [42]:
count_obs_df = obs_mut.groupby(['Species', 'Gene']).Mut3.value_counts()
count_obs_df.name = 'CountMut3'
count_obs_df = count_obs_df.reset_index()
final_obs = count_obs_df.pivot(index=['Species', 'Gene'], columns='Mut3', values='CountMut3').fillna(0)
final_obs['ExpectedObserved'] = 'Observed'

In [43]:
count_exp_df = exp_mut.groupby(['Species', 'Gene']).Mut3.value_counts()
count_exp_df.name = 'CountMut3'
count_exp_df = count_exp_df.reset_index()
final_exp = count_exp_df.pivot(index=['Species', 'Gene'], columns='Mut3', values='CountMut3').fillna(0)
final_exp['ExpectedObserved'] = 'Expected'

In [44]:
final_exp.head()

,Mut3,A[A>G]A,A[A>G]C,A[A>G]G,A[A>G]T,A[C>T]A,A[C>T]C,A[C>T]G,A[C>T]T,A[G>A]A,A[G>A]C,...,T[T>A]T,T[T>C]A,T[T>C]C,T[T>C]G,T[T>C]T,T[T>G]A,T[T>G]C,T[T>G]G,T[T>G]T,ExpectedObserved
Species,Gene,,,,,,,,,,,,,,,,,,,,,
Abbottina_obtusirostris,CYTB,8.0,4.0,4.0,4.0,12.0,10.0,10.0,17.0,0.0,0.0,...,2.0,10.0,14.0,12.0,10.0,3.0,8.0,6.0,2.0,Expected
Abbottina_rivularis,CYTB,6.0,3.0,3.0,3.0,16.0,11.0,9.0,21.0,2.0,2.0,...,1.0,5.0,9.0,8.0,12.0,4.0,3.0,3.0,1.0,Expected
Abramis_brama,CYTB,9.0,3.0,4.0,5.0,8.0,9.0,9.0,18.0,0.0,1.0,...,2.0,10.0,15.0,13.0,11.0,4.0,4.0,6.0,2.0,Expected
Acanthogobius_hasta,CYTB,7.0,2.0,3.0,2.0,12.0,10.0,9.0,14.0,2.0,2.0,...,3.0,14.0,16.0,22.0,15.0,5.0,9.0,12.0,3.0,Expected
Acanthopagrus_latus,CYTB,8.0,3.0,4.0,3.0,10.0,12.0,7.0,13.0,0.0,3.0,...,3.0,11.0,12.0,18.0,17.0,5.0,5.0,9.0,3.0,Expected


### Merge tables and sort by species

In [90]:
final_df = pd.concat([final_exp, final_obs])
final_df = final_df.sort_values(['Species', 'Gene'])
final_df.insert(0, 'ExpectedObserved', final_df.pop('ExpectedObserved'))
final_df = final_df.fillna(0)
final_df = final_df.reset_index()

/var/folders/5l/9jtg_90x5s575m1zqn_h0kgm0000gn/T/ipykernel_56417/533273452.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  final_df.insert(0, 'ExpectedObserved', final_df.pop('ExpectedObserved'))


### Add 0 to non presented cont

In [91]:
possible_sbs192 = [
    "A[A>C]A", "A[A>C]C", "A[A>C]G", "A[A>C]T", "C[A>C]A", "C[A>C]C", "C[A>C]G", "C[A>C]T", 
    "G[A>C]A", "G[A>C]C", "G[A>C]G", "G[A>C]T", "T[A>C]A", "T[A>C]C", "T[A>C]G", "T[A>C]T", 
    "A[A>G]A", "A[A>G]C", "A[A>G]G", "A[A>G]T", "C[A>G]A", "C[A>G]C", "C[A>G]G", "C[A>G]T", 
    "G[A>G]A", "G[A>G]C", "G[A>G]G", "G[A>G]T", "T[A>G]A", "T[A>G]C", "T[A>G]G", "T[A>G]T", 
    "A[A>T]A", "A[A>T]C", "A[A>T]G", "A[A>T]T", "C[A>T]A", "C[A>T]C", "C[A>T]G", "C[A>T]T", 
    "G[A>T]A", "G[A>T]C", "G[A>T]G", "G[A>T]T", "T[A>T]A", "T[A>T]C", "T[A>T]G", "T[A>T]T", 
    "A[C>A]A", "A[C>A]C", "A[C>A]G", "A[C>A]T", "C[C>A]A", "C[C>A]C", "C[C>A]G", "C[C>A]T", 
    "G[C>A]A", "G[C>A]C", "G[C>A]G", "G[C>A]T", "T[C>A]A", "T[C>A]C", "T[C>A]G", "T[C>A]T", 
    "A[C>G]A", "A[C>G]C", "A[C>G]G", "A[C>G]T", "C[C>G]A", "C[C>G]C", "C[C>G]G", "C[C>G]T", 
    "G[C>G]A", "G[C>G]C", "G[C>G]G", "G[C>G]T", "T[C>G]A", "T[C>G]C", "T[C>G]G", "T[C>G]T", 
    "A[C>T]A", "A[C>T]C", "A[C>T]G", "A[C>T]T", "C[C>T]A", "C[C>T]C", "C[C>T]G", "C[C>T]T", 
    "G[C>T]A", "G[C>T]C", "G[C>T]G", "G[C>T]T", "T[C>T]A", "T[C>T]C", "T[C>T]G", "T[C>T]T", 
    "A[G>A]A", "A[G>A]C", "A[G>A]G", "A[G>A]T", "C[G>A]A", "C[G>A]C", "C[G>A]G", "C[G>A]T", 
    "G[G>A]A", "G[G>A]C", "G[G>A]G", "G[G>A]T", "T[G>A]A", "T[G>A]C", "T[G>A]G", "T[G>A]T", 
    "A[G>C]A", "A[G>C]C", "A[G>C]G", "A[G>C]T", "C[G>C]A", "C[G>C]C", "C[G>C]G", "C[G>C]T", 
    "G[G>C]A", "G[G>C]C", "G[G>C]G", "G[G>C]T", "T[G>C]A", "T[G>C]C", "T[G>C]G", "T[G>C]T", 
    "A[G>T]A", "A[G>T]C", "A[G>T]G", "A[G>T]T", "C[G>T]A", "C[G>T]C", "C[G>T]G", "C[G>T]T", 
    "G[G>T]A", "G[G>T]C", "G[G>T]G", "G[G>T]T", "T[G>T]A", "T[G>T]C", "T[G>T]G", "T[G>T]T", 
    "A[T>A]A", "A[T>A]C", "A[T>A]G", "A[T>A]T", "C[T>A]A", "C[T>A]C", "C[T>A]G", "C[T>A]T", 
    "G[T>A]A", "G[T>A]C", "G[T>A]G", "G[T>A]T", "T[T>A]A", "T[T>A]C", "T[T>A]G", "T[T>A]T", 
    "A[T>C]A", "A[T>C]C", "A[T>C]G", "A[T>C]T", "C[T>C]A", "C[T>C]C", "C[T>C]G", "C[T>C]T", 
    "G[T>C]A", "G[T>C]C", "G[T>C]G", "G[T>C]T", "T[T>C]A", "T[T>C]C", "T[T>C]G", "T[T>C]T", 
    "A[T>G]A", "A[T>G]C", "A[T>G]G", "A[T>G]T", "C[T>G]A", "C[T>G]C", "C[T>G]G", "C[T>G]T", 
    "G[T>G]A", "G[T>G]C", "G[T>G]G", "G[T>G]T", "T[T>G]A", "T[T>G]C", "T[T>G]G", "T[T>G]T", 
]

In [92]:
for sbs in possible_sbs192:
    if sbs not in final_df.columns:
        final_df[sbs] = 0

In [94]:
final_df.to_csv('../data/counted_codons_cytb.csv')